# Figure 4

In [48]:
%run 'stability_fig_init.py'

In [52]:
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.01, shared_yaxes=True)
color = px.colors.qualitative.T10[0]

kendall = []
for i, area in enumerate(["GB", "Nordic"]):
    target = "f_rocof"
    feature = "CE_import_export_total_ramp"

    with open(
        fit_folder.format(area, version, target) + "shap_values_gtb_full.pkl", "rb"
    ) as handle:
        sh = pickle.load(handle)
    X_test = pd.read_hdf(version_folder.format(area, version) + "X_test_full.h5")

    feature_ind = np.where(X_test.columns == feature)[0][0]
    fig.add_trace(
        go.Scatter(
            x=X_test[feature] / 1000,
            y=sh.values[:, feature_ind] * 1000,
            mode="markers",
            marker_color=color,
            marker_size=4,
            marker_opacity=0.75,
        ),
        row=1,
        col=i + 1,
    )
    kendall.append(
        pd.Series(X_test.reset_index()[feature]).corr(
            pd.Series(sh.values[:, feature_ind]), method="kendall"
        )
    )
fig.update_layout(
    {
        "yaxis{}".format(1): {
            "tickmode": "array",
            "tickvals": [-0.4, -0.2, 0, 0.2, 0.4],
            "title_standoff": 200,
            "range": [-0.47, 0.47],
            "title": "SHAP values of RoCoF [mHz/s] <br> <i>inflow grid</i>",
            "linewidth": 1,
            "linecolor": "black",
            "mirror": True,
            "ticks": "outside",
            "showline": True,
        },
        "xaxis{}".format(1): {
            "title_standoff": 200,
            "range": [-1, 1],
            "title": "CE to GB scheduled flow ramp [GW/h]",
            "linewidth": 1,
            "linecolor": "black",
            "mirror": True,
            "ticks": "outside",
            "showline": True,
        },
    }
)
fig.update_layout(
    {
        "yaxis{}".format(2): {
            "tickmode": "array",
            "tickvals": [-0.4, -0.2, 0, 0.2, 0.4],
            "title_standoff": 200,
            "linewidth": 1,
            "linecolor": "black",
            "mirror": True,
            "ticks": "outside",
            "showline": True,
        },
        "xaxis{}".format(2): {
            "title_standoff": 200,
            "title": "CE to Nordic scheduled flow ramp [GW/h]",
            "linewidth": 1,
            "linecolor": "black",
            "mirror": True,
            "ticks": "outside",
            "showline": True,
        },
    }
)

fig.update_layout(font=dict(size=20, color="black"))
fig.update_annotations(font_size=20)
fig.update_layout(margin_t=0)
fig.update_layout(margin_r=50)
fig.update_layout(template="plotly_white", width=1240, height=500, showlegend=False)

fig.add_annotation(
    x=0.05,
    y=0.97,
    text="&#964; = {}".format(np.round(kendall[0], 2)),
    showarrow=False,
    xanchor="left",
    xref="x domain",
    yref="y domain",
    font_size=30,
)
fig.add_annotation(
    x=0.05,
    y=0.97,
    text="&#964; = {}".format(np.round(kendall[1], 2)),
    showarrow=False,
    xanchor="left",
    xref="x2 domain",
    yref="y2 domain",
    font_size=30,
)

print("creating plot...")
scope = pio.kaleido.scope
scope._shutdown_kaleido()
pio.write_image(fig, "fig4.pdf", format="pdf", validate=False, engine="kaleido")
time.sleep(1)
pio.write_image(fig, "fig4.pdf", format="pdf", validate=False, engine="kaleido")
scope._shutdown_kaleido()


creating plot...


# Calculating Relative Ramping Speeds

In [49]:
#### Select important generation ramps for the RoCoF ####
ramps = np.array(
    [
        "nuclear_ramp",
        "pumped_hydro_ramp",
        "reservoir_hydro_ramp",
        "gas_ramp",
        "hard_coal_ramp",
        "lignite_ramp",
        "fossil_peat_ramp",
        "CE_import_export_total_ramp",
        "GB_import_export_total_ramp",
        "Nordic_import_export_total_ramp",
    ]
)


#### Calculate relative ramping speeds ####
rel_ramp_speed = pd.DataFrame(index=areas, columns=ramps)
ramp_rate = pd.DataFrame(index=areas, columns=ramps)
mean_ramp = pd.DataFrame(index=areas, columns=ramps)

ramp_rate.loc["Nordic", "CE_import_export_total_ramp"] = 0.017
ramp_rate.loc["GB", "CE_import_export_total_ramp"] = 10 / 100
ramp_rate.loc["CE", "GB_import_export_total_ramp"] = 10 / 100
ramp_rate.loc["CE", "Nordic_import_export_total_ramp"] = 0.017


for i, area in enumerate(areas):

    # Load results and data
    X_test_act = pd.read_hdf(version_folder.format(area, version) + "X_test_full.h5")
    shap_vals_act = shap_vals_df(area, version, "f_rocof", model_type="_full")
    inputs = pd.read_hdf(data_folder.format(area) + "input_actual.h5")

    # Define ramping rates
    ramp_rate.loc[area, "nuclear_ramp"] = 2 / 100
    ramp_rate.loc[area, "gas_ramp"] = (
        15 / 100
    )  # 20 for OCGT or 8 for CCGT, so we take a value in between
    ramp_rate.loc[area, "pumped_hydro_ramp"] = 15 / 100  # same as for reservoir hydro
    ramp_rate.loc[area, "reservoir_hydro_ramp"] = 15 / 100
    ramp_rate.loc[area, "hard_coal_ramp"] = 6 / 100
    ramp_rate.loc[area, "lignite_ramp"] = 4 / 100
    ramp_rate.loc[area, "fossil_peat_ramp"] = 8 / 100

    for ramp in ramps:
        if ramp in shap_vals_act.columns:
            # Determine mean ramp
            mean_ramp.loc[area, ramp] = inputs.loc[:, ramp].abs().mean()

# Calculate proxy for ramping speed
ramp_speed = ramp_rate * mean_ramp
rel_ramp_speed = (ramp_speed.T / ramp_speed.max(axis=1)).T
rel_ramp_speed = rel_ramp_speed.dropna(axis=1, how="all")


In [50]:
rel_ramp_speed

,nuclear_ramp,pumped_hydro_ramp,reservoir_hydro_ramp,gas_ramp,hard_coal_ramp,lignite_ramp,fossil_peat_ramp,CE_import_export_total_ramp,GB_import_export_total_ramp,Nordic_import_export_total_ramp
CE,0.028937,0.549529,0.654799,1.0,0.244784,0.093161,NaN,NaN,0.093294,0.029237
Nordic,0.001841,NaN,1.0,0.018895,0.011502,NaN,0.006791,0.0413,NaN,NaN
GB,0.004522,0.146715,NaN,1.0,0.103979,NaN,NaN,0.140278,NaN,NaN
